<a href="https://colab.research.google.com/github/YRLT/Experiment_result_save/blob/main/%E2%80%9CEvaluating_position_of_TAPAS_on_the_Tabfact_test_set_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!add-apt-repository ppa:alessandro-strada/google-drive-ocamlfuse-beta
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Fatal Python error: initsite: Failed to import the site module
Traceback (most recent call last):
  File "/usr/lib/python3.7/site.py", line 596, in <module>
Traceback (most recent call last):
  File "/usr/bin/add-apt-repository", line 109, in <module>
    distro = aptsources.distro.get_distro()
    main()
  File "/usr/lib/python3.7/site.py", line 583, in main
    known_paths = addsitepackages(known_paths)
  File "/usr/lib/python3/dist-packages/aptsources/distro.py", line 590, in get_distro
  File "/usr/lib/python3.7/site.py", line 366, in addsitepackages
    addsitedir(sitedir, known_paths)
    lsb_result = _lsb_release()
  File "/usr/lib/python3/dist-packages/aptsources/distro.py", line 479, in _lsb_release
    out = Popen(['lsb_release', '-idrc'], stdout=PIPE).communicate()[0]
  File "/usr/lib/python3.7/site.py", line 213, in addsitedir
    addpackage(sitedir, name, known_paths)
  File "/usr/lib/python3.6/subproces

In [ ]:
! google-drive-ocamlfuse -cc

Clearing cache...done


In [ ]:
!echo $'#!/bin/sh\necho $* > /dev/stderr' > xdg-open
!chmod 755 xdg-open
!env PATH=`pwd`:$PATH google-drive-ocamlfuse


## Introduction

In this notebook, we are going to run `TapasForSequenceClassification`, a PyTorch/Transformers implementation of the [Tapas algorithm](https://arxiv.org/abs/2004.02349) by Google AI on the test set of [TabFact](https://github.com/wenhuchen/Table-Fact-Checking), a large dataset for table entailment (which is included in the HuggingFace [datasets library](https://github.com/huggingface/datasets)). In this way, we can verify whether our implementation is consistent with the results reported in the paper.

* Paper (which is a follow-up on the original TAPAS paper): https://arxiv.org/abs/2010.00571
* Tabfact paper: https://arxiv.org/abs/1909.02164

Note that `TapasForSequenceClassification` is really similar to `BertForSequenceClassification` (i.e. it adds a linear layer on top of the pooled output). The difference between the two is that for TAPAS, you need to use `TapasTokenizer` to prepare table-question pairs for the model instead of regular sequences.

## Setting up environment

Make sure to set runtime to GPU.
We install the `transformers` library from source, as well as the soft dependency on `torch-scatter`:

In [ ]:
! rm -r transformers
! git clone https://github.com/huggingface/transformers.git
! cd transformers
! pip install ./transformers

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 67076 (delta 22), reused 19 (delta 5), pack-reused 67021
Receiving objects: 100% (67076/67076), 50.00 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (47649/47649), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 4.3MB/s 
     |████████████████████████████████| 3.2MB 39.9MB/s 
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=1972345 sha256=3ef301a3363751874994d7955e6c3b08c040961f4bfd9d9583989e1565892011
  Stored in directory: /tmp/pip-ephem-wheel-cache-hgkk34qn/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers

In [ ]:
! pip install torch-scatter==2.0.6 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

import torch

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 4.4MB/s 


We also install the datasets library from source:

In [ ]:
! rm -r datasets
! git clone https://github.com/huggingface/datasets.git
! cd datasets
! pip install ./datasets

rm: cannot remove 'datasets': No such file or directory
Cloning into 'datasets'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 34432 (delta 63), reused 38 (delta 13), pack-reused 34290
Receiving objects: 100% (34432/34432), 49.30 MiB | 18.27 MiB/s, done.
Resolving deltas: 100% (13139/13139), done.
Processing ./datasets
     |████████████████████████████████| 245kB 4.1MB/s 
     |████████████████████████████████| 112kB 15.3MB/s 
  Created wheel for datasets: filename=datasets-1.4.1.dev0-cp37-none-any.whl size=189541 sha256=ae1b1a013a55b2d4a9b64defb6540881cc88956a00f665b01fcfe96b30145523
  Stored in directory: /tmp/pip-ephem-wheel-cache-0cdv6e6o/wheels/d7/33/59/8f64453f60990c3158cc14272127d74a1f77d9919b010387d1
Successfully built datasets


In [ ]:
import torch
torch.__version__

'1.8.0+cu101'

In [ ]:
! nvidia-smi
!sudo lsb_release -a

Thu Mar 18 13:37:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!sudo fuser /dev/nvidia*
# !kill -9 1226

## Loading the model

Here we load in a base-sized TAPAS model, which was fine-tuned on TabFact, and move it to GPU (if available):




In [ ]:
from transformers import TapasForSequenceClassification
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")
model.to(device)

TapasForSequenceClassification(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.07, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0): TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=T

## Preparing the data

Here we read in the test set of the TabFact dataset, on which we are going to evaluate the fine-tuned checkpoint.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('tab_fact', 'tab_fact', split='test')

Dataset tab_fact downloaded and prepared to /root/.cache/huggingface/datasets/tab_fact/tab_fact/1.0.0/bd64c4ee1b4127f8377f1817669219ec36aaf65cb8c78d7c995902e25ef362b6. Subsequent calls will reuse this data.


In [ ]:
dataset.column_names

['id', 'label', 'statement', 'table_caption', 'table_id', 'table_text']

Each example in the TabFact dataset is a statement about a table, and the label indicates whether the statement is supported (1) or refuted (0) by the contents of the table. So it's a binary classification problem. 

Let's visualize an example:

In [ ]:
import pandas as pd

# let's take a random example
example = dataset[0]
id2label = {0: "REFUTES", 1: "SUPPORTS"}

data = example['table_text']

# convert table_text into a Pandas dataframe
table1 = pd.DataFrame([x.split('#') for x in data.split('\n')[1:-2]], columns=[x for x in data.split('\n')[0].split('#')])
table2 = pd.DataFrame([x.split('#') for x in data.split('\n')[-2:-1]], columns=[x for x in data.split('\n')[0].split('#')])
display(table1)
display(table2)
table = pd.concat([table1,table2],axis = 0)
display(table)
print("Statement:", example['statement'])
print("Label:", id2label[example['label']])

,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,totals,1,5,8,12,18,16


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
0,totals,1,5,8,12,18,16


Statement: tony lema be in the top 5 for the master tournament , the us open , and the open championship
Label: SUPPORTS


We write the logic to turn the `table_text` column into a Pandas dataframe into a function:

In [ ]:
def read_text_as_pandas_table(table_text: str):
    table = pd.DataFrame([x.split('#') for x in table_text.split('\n')[1:-1]], columns=[x for x in table_text.split('\n')[0].split('#')]).fillna('')
    table = table.astype(str)
    display(table)
    return table

In [ ]:
def read_text_as_pandas_table_with_shuffle0(table_text: str):
    table1 = pd.DataFrame([x.split('#') for x in table_text.split('\n')[1:-2]], columns=[x for x in table_text.split('\n')[0].split('#')]).fillna('')
    table1 = table1.sample(frac = 1, axis = 0)
    table2 = pd.DataFrame([x.split('#') for x in data.split('\n')[-2:-1]], columns=[x for x in data.split('\n')[0].split('#')])
    table = pd.concat([table1,table2],axis=0,ignore_index = True)
    table.reset_index(drop=True)
    # display(table)
    table = table.astype(str)
    return table

In [ ]:
def read_text_as_pandas_table_with_shuffle1(table_text: str):
    table = pd.DataFrame([x.split('#') for x in table_text.split('\n')[1:-1]], columns=[x for x in table_text.split('\n')[0].split('#')]).fillna('')
    table = table.sample(frac = 1,axis = 1)
    table = table.astype(str)
    return table

In [ ]:
def read_text_as_pandas_table_with_shuffle01(table_text: str):
    table1 = pd.DataFrame([x.split('#') for x in table_text.split('\n')[1:-2]], columns=[x for x in table_text.split('\n')[0].split('#')]).fillna('')
    table1 = table1.sample(frac = 1, axis = 0)
    table2 = pd.DataFrame([x.split('#') for x in data.split('\n')[-2:-1]], columns=[x for x in data.split('\n')[0].split('#')])
    table = pd.concat([table1,table2],axis=0,ignore_index = True)
    table.reset_index(drop=True)
    table = table.sample(frac = 1,axis = 1)
    table = table.astype(str)
    return table

Let's check if TapasTokenizer can prepare the data correctly:

In [ ]:
from transformers import TapasTokenizer

tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-tabfact")

# test on a random example
example = dataset[0]
#inputs = tokenizer(table=read_text_as_pandas_table(example['table_text']),
inputs = tokenizer(table=read_text_as_pandas_table(example['table_text']),
                   queries=example['statement'],
                   padding='max_length')
inputs

,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


{'input_ids': [101, 4116, 3393, 2863, 2022, 1999, 1996, 2327, 1019, 2005, 1996, 3040, 2977, 1010, 1996, 2149, 2330, 1010, 1998, 1996, 2330, 2528, 102, 2977, 5222, 2327, 1011, 1019, 2327, 1011, 2184, 2327, 1011, 2423, 2824, 7659, 2081, 5972, 2977, 1014, 1015, 1016, 1018, 1018, 1018, 2149, 2330, 1014, 1016, 1017, 1018, 1020, 1019, 1996, 2330, 2528, 1015, 1016, 1016, 1016, 1017, 1017, 14198, 2528, 1014, 1014, 1015, 1016, 1019, 1018, 21948, 1015, 1019, 1022, 2260, 2324, 2385, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Now let's use the `.map()` functionality of `datasets` to tokenize and prepare for the model the entire test split of the dataset. Note that we tokenize each table-question pair independently (we don't set `batched=True`): 

In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D

# we need to define the features ourselves as the token_type_ids of TAPAS are different from those of BERT 
features = Features({
    'attention_mask': Sequence(Value(dtype='int64')),
    'id': Value(dtype='int32'),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'label': ClassLabel(names=['refuted', 'entailed']),
    'statement': Value(dtype='string'),
    'table_caption': Value(dtype='string'),
    'table_id': Value(dtype='string'),
    'table_text': Value(dtype='string'),
    'token_type_ids': Array2D(dtype="int64", shape=(512, 7))
})
test = dataset.map(
    lambda e: tokenizer(table=read_text_as_pandas_table(e['table_text']), queries=e['statement'], 
                                       truncation=True,
                                       padding='max_length'),
    features=features
)

,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


,tournament,wins,top - 5,top - 10,top - 25,events,cuts made
0,masters tournament,0,1,2,4,4,4
1,us open,0,2,3,4,6,5
2,the open championship,1,2,2,2,3,3
3,pga championship,0,0,1,2,5,4
4,totals,1,5,8,12,18,16


...

,game,date,team,location attendance,record
0,75,april 1,houston,us airways center 18422,41 - 34
1,76,april 3,sacramento,us airways center 18422,42 - 34
2,77,april 5,dallas,american airlines center 20301,42 - 35
3,78,april 8,new orleans,new orleans arena 17781,43 - 35
4,79,april 10,memphis,fedexforum 15908,43 - 36
5,80,april 11,minnesota,target center 18478,44 - 36
6,81,april 13,memphis,us airways center 18422,45 - 36
7,82,april 15,golden state,us airways center,46 - 36


,game,date,team,location attendance,record
0,75,april 1,houston,us airways center 18422,41 - 34
1,76,april 3,sacramento,us airways center 18422,42 - 34
2,77,april 5,dallas,american airlines center 20301,42 - 35
3,78,april 8,new orleans,new orleans arena 17781,43 - 35
4,79,april 10,memphis,fedexforum 15908,43 - 36
5,80,april 11,minnesota,target center 18478,44 - 36
6,81,april 13,memphis,us airways center 18422,45 - 36
7,82,april 15,golden state,us airways center,46 - 36


,rank,name,nation,sp + fs,points,places
0,1,linda fratianne,united states,1,186.92,11
1,2,anett pötzsch,east germany,3,184.36,18
2,3,emi watanabe,japan,4,180.52,31
3,4,dagmar lurz,west germany,6,179.96,33
4,5,denise biellmann,switzerland,2,177.28,49
5,6,lisa - marie allen,united states,5,176.68,54
6,7,claudia kristofics - binder,austria,7,175.44,63
7,8,susanna driano,italy,9,173.46,70
8,9,carola weißenberg,east germany,11,170.54,88
9,10,kristiina wegelius,finland,15,169.26,98


,rank,name,nation,sp + fs,points,places
0,1,linda fratianne,united states,1,186.92,11
1,2,anett pötzsch,east germany,3,184.36,18
2,3,emi watanabe,japan,4,180.52,31
3,4,dagmar lurz,west germany,6,179.96,33
4,5,denise biellmann,switzerland,2,177.28,49
5,6,lisa - marie allen,united states,5,176.68,54
6,7,claudia kristofics - binder,austria,7,175.44,63
7,8,susanna driano,italy,9,173.46,70
8,9,carola weißenberg,east germany,11,170.54,88
9,10,kristiina wegelius,finland,15,169.26,98


KeyboardInterrupt: ignored

Let's create a PyTorch dataloader based on this:

In [ ]:
# map to PyTorch tensors and only keep columns we need
test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])
# create PyTorch dataloader
test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

We can verify whether everything is created correctly, for example by verifying their shapes and decoding the `input_ids` of the first example of the first batch:

In [ ]:
# let's check the first batch
batch = next(iter(test_dataloader))
#assert batch["input_ids"].shape == (4, 512)
#assert batch["attention_mask"].shape == (4, 512)
#assert batch["token_type_ids"].shape == (4, 512, 7)
tokenizer.decode(batch["input_ids"][0])

'[CLS] tony lema be in the top 5 for the master tournament, the us open, and the open championship [SEP] tournament wins top - 5 top - 10 top - 25 events cuts made masters tournament 0 1 2 4 4 4 us open 0 2 3 4 6 5 the open championship 1 2 2 2 3 3 pga championship 0 0 1 2 5 4 totals 1 5 8 12 18 16 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

## Run evaluation

Now we can compute the accuracy of TAPAS on the test set of TabFact! Incredible how easy 🤗 datasets makes this!

Note that this will take a couple of minutes. We set the batch size to only 4, to make sure a single GPU on Google Colab can handle this. You can increase it of course.

In [ ]:
from datasets import load_metric

accuracy = load_metric("accuracy")

print("Starting evaluation...")
number_processed = 0
total = len(test_dataloader) * batch["input_ids"].shape[0] # number of batches * batch_size

for batch in test_dataloader:
    # get the inputs
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    token_type_ids = batch["token_type_ids"].to(device)
    labels = batch["label"].to(device)

    # forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
    model_predictions = outputs.logits.argmax(-1)

    # add metric
    accuracy.add_batch(predictions=model_predictions, references=labels)

    number_processed += batch["input_ids"].shape[0]
    if number_processed % 100 == 0:
      print(f"Processed {number_processed} / {total} examples")


final_score = accuracy.compute()

Starting evaluation...
Processed 200 / 12784 examples
Processed 400 / 12784 examples
Processed 600 / 12784 examples
Processed 800 / 12784 examples
Processed 1000 / 12784 examples
Processed 1200 / 12784 examples
Processed 1400 / 12784 examples
Processed 1600 / 12784 examples
Processed 1800 / 12784 examples
Processed 2000 / 12784 examples
Processed 2200 / 12784 examples
Processed 2400 / 12784 examples
Processed 2600 / 12784 examples
Processed 2800 / 12784 examples
Processed 3000 / 12784 examples
Processed 3200 / 12784 examples
Processed 3400 / 12784 examples
Processed 3600 / 12784 examples
Processed 3800 / 12784 examples
Processed 4000 / 12784 examples
Processed 4200 / 12784 examples
Processed 4400 / 12784 examples
Processed 4600 / 12784 examples
Processed 4800 / 12784 examples
Processed 5000 / 12784 examples
Processed 5200 / 12784 examples
Processed 5400 / 12784 examples
Processed 5600 / 12784 examples
Processed 5800 / 12784 examples
Processed 6000 / 12784 examples
Processed 6200 / 1278

In [ ]:
!google-drive-ocamlfuse -cc

Clearing cache...done


# 新段落

In [ ]:
print("oringnal result:",final_score) #5000sample

oringnal result: {'accuracy': 0.715055910543131}


In [ ]:
print("zero embedding",final_score)

zero embedding {'accuracy': 0.590055910543131}


In [ ]:
print("5 times embedding",final_score)

5 times embedding {'accuracy': 0.5015974440894568}


In [ ]:
print("1.5 times embedding",final_score)

1.5 times embedding {'accuracy': 0.709464856230032}


In [ ]:
print("1.2 times embedding",final_score)

1.2 times embedding {'accuracy': 0.7138578274760383}


In [ ]:
print("1.1 times embedding",final_score)

1.1 times embedding {'accuracy': 0.7160543130990416}


In [ ]:
print("1.15 times embedding",final_score)

1.15 times embedding {'accuracy': 0.7140575079872205}


In [ ]:
print("result of shuflle columns",final_score)

result of shuflle columns {'accuracy': 0.7038738019169329}


In [ ]:
print("result of shuflle rows",final_score)

result of shuflle rows {'accuracy': 0.6721246006389776}


In [ ]:
print("result of shuflle cols&rows",final_score)

result of shuflle cols&rows {'accuracy': 0.6669329073482428}


In [ ]:
print("result of shuflle cols&rows with all samples",final_score)

result of shuflle cols&rows with all samples {'accuracy': 0.7197746302527585}


In [ ]:
print("result of shuflle cols with all samples",final_score)

result of shuflle cols with all samples {'accuracy': 0.7653181000078253}


In [ ]:
print("result of shuflle rows with all samples",final_score)

result of shuflle rows with all samples {'accuracy': 0.7229830190155724}


In [ ]:
print("oringinal result with all samples",final_score)

oringinal result with all samples {'accuracy': 0.7711871038422412}
